In [ ]:
from dask.distributed import Client

client = Client("YOUR_CLIENT")
client

In [2]:
import yaml
import numpy as np
from collections import defaultdict

import awkward as ak
import uproot
import coffea
import coffea.processor as processor
from coffea import hist

from coffea.nanoevents import NanoEventsFactory
from coffea.nanoevents import NanoAODSchema

import matplotlib.pyplot as plt

In [3]:
year = '2016postVFP'


with open("HLT_Run2_UL.yaml", 'r') as f_yml:
    dict_HLT = yaml.load(f_yml, Loader=yaml.FullLoader)

hlt_ls = [_hlt.split('HLT_')[-1] for _hlt_ls in dict_HLT[year[:4]].values() for _hlt in _hlt_ls]

with open(f'{year}_MC.txt') as f_tt:
    lines = f_tt.read().splitlines()

fileset_TT = [''.join(['root://xcache/', line]) for line in lines]

fileset_MC_Data = {'TT': fileset_TT}

In [5]:
class Trig_Processor(processor.ProcessorABC):
    def __init__(self, hlts_lep, era=2018):
        self.era = era
        self.hlts_lep = hlts_lep
        
        dataset_axis = hist.Cat("dataset", "")

        # 2.4 because of b-jet cuts, 1.5 because of ECAL region
        bins_pt = [30, 50, 100, 200, 400, 800]
        bins_eta = [-2.4, -1.5, -0.7, 0, 0.7, 1.5, 2.4]

        axis_pt = hist.Bin("pT", "pT [GeV]", bins_pt)
        axis_eta = hist.Bin("eta", "eta [AU]", bins_eta)
        
        self._accumulator = processor.dict_accumulator({
            # bottom
            'h_num_b': hist.Hist('h_num_b', dataset_axis, axis_pt, axis_eta),
            'h_den_b': hist.Hist('h_den_b', dataset_axis, axis_pt, axis_eta),
            # charm
            'h_num_c': hist.Hist('h_num_c', dataset_axis, axis_pt, axis_eta),
            'h_den_c': hist.Hist('h_den_c', dataset_axis, axis_pt, axis_eta),
            # light
            'h_num_l': hist.Hist('h_num_l', dataset_axis, axis_pt, axis_eta),
            'h_den_l': hist.Hist('h_den_l', dataset_axis, axis_pt, axis_eta),
        })


    @property
    def accumulator(self):
        return self._accumulator


    def postprocess(self, accumulator):
        return accumulator
    

    def flt_HLT(self, events, hlt_avail):
        hlt_good = [hlt for hlt in self.hlts_lep if hlt in hlt_avail]
        mask_hlt = eval(' | '.join(([f'events.HLT.{hlt}' for hlt in hlt_good])))
        events_flt = events[mask_hlt]

        return events_flt


    def get_good_jets(self, jets):
        jets = jets[jets.jetId == 6]
        jets = jets[jets.pt >= 30.]
        jets = jets[abs(jets.eta) < 4.7]
        return jets


    def btag_id(self, wp):
        # using deepjet
        # ref : https://twiki.cern.ch/twiki/bin/view/CMS/BtagRecommendation
        dict_wp = {"2016preVFP": {"loose": 0.0508, "medium": 0.2598, "tight": 0.6502},
                   "2016postVFP": {"loose": 0.0480, "medium": 0.2489, "tight": 0.6377},
                   "2017": {"loose": 0.0532, "medium": 0.3040, "tight": 0.7476},
                   "2018": {"loose": 0.0490, "medium": 0.2783, "tight": 0.7100},}
        return dict_wp[self.era][wp]


    def process(self, events):
        output = self.accumulator.identity()
        dataset = events.metadata["dataset"]
        
        # HLT
        hlt_avail = events.HLT.layout.keys()
        events = self.flt_HLT(events, hlt_avail)

        # good jets
        jets = self.get_good_jets(events.Jet)
        b_jets = jets[jets.hadronFlavour==5]
        c_jets = jets[jets.hadronFlavour==4]
        l_jets = jets[jets.hadronFlavour==0]
        
        # tagged jets
        wp = self.btag_id('loose')
        b_btag_jets = b_jets[b_jets.btagDeepFlavB>wp]
        c_btag_jets = c_jets[c_jets.btagDeepFlavB>wp]
        l_btag_jets = l_jets[l_jets.btagDeepFlavB>wp]

        # get pT and eta
        b_num_pt = ak.flatten(b_btag_jets.pt, axis=None)
        b_num_eta = ak.flatten(b_btag_jets.eta, axis=None)
        b_den_pt = ak.flatten(b_jets.pt, axis=None)
        b_den_eta = ak.flatten(b_jets.eta, axis=None)

        c_num_pt = ak.flatten(c_btag_jets.pt, axis=None)
        c_num_eta = ak.flatten(c_btag_jets.eta, axis=None)
        c_den_pt = ak.flatten(c_jets.pt, axis=None)
        c_den_eta = ak.flatten(c_jets.eta, axis=None)

        l_num_pt = ak.flatten(l_btag_jets.pt, axis=None)
        l_num_eta = ak.flatten(l_btag_jets.eta, axis=None)
        l_den_pt = ak.flatten(l_jets.pt, axis=None)
        l_den_eta = ak.flatten(l_jets.eta, axis=None)

        # fill histograms
        output['h_num_b'].fill(dataset=dataset, pT=b_num_pt, eta=b_num_eta)
        output['h_den_b'].fill(dataset=dataset, pT=b_den_pt, eta=b_den_eta)

        output['h_num_c'].fill(dataset=dataset, pT=c_num_pt, eta=c_num_eta)
        output['h_den_c'].fill(dataset=dataset, pT=c_den_pt, eta=c_den_eta)

        output['h_num_l'].fill(dataset=dataset, pT=l_num_pt, eta=l_num_eta)
        output['h_den_l'].fill(dataset=dataset, pT=l_den_pt, eta=l_den_eta)

        return output



In [6]:
executor = processor.DaskExecutor(client=client)

run = processor.Runner(executor=executor,
                       schema=processor.NanoAODSchema,
                       savemetrics=True,
                       chunksize=10000*1000)

output, metrics = run(fileset=fileset_MC_Data,
                      treename="Events",
                      processor_instance=Trig_Processor(era=year, hlts_lep=hlt_ls),)

metrics

{'bytesread': 2178673866,
 'columns': ['HLT_IsoTkMu24',
  'nJet',
  'HLT_IsoMu20',
  'HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL_DZ',
  'HLT_Ele27_WPTight_Gsf',
  'HLT_IsoMu24',
  'HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ',
  'HLT_IsoTkMu22',
  'Jet_jetId',
  'HLT_Mu17_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL',
  'HLT_Mu17_TrkIsoVVL_TkMu8_TrkIsoVVL_DZ',
  'HLT_Ele35_WPLoose_Gsf',
  'HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL',
  'Jet_pt',
  'Jet_hadronFlavour',
  'HLT_Ele27_eta2p1_WPLoose_Gsf',
  'HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL',
  'Jet_btagDeepFlavB',
  'HLT_Mu17_TrkIsoVVL_TkMu8_TrkIsoVVL',
  'HLT_Ele25_eta2p1_WPTight_Gsf',
  'HLT_IsoTkMu20',
  'HLT_IsoMu22',
  'HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL',
  'HLT_Mu8_TrkIsoVVL_Ele17_CaloIdL_TrackIdL_IsoVL',
  'Jet_eta'],
 'entries': 43546000,
 'processtime': 3400.131105899811,
 'chunks': 49}

In [7]:
import pickle

with open(f'btag_output_{year}.pickle', 'wb') as handle:
    pickle.dump(output, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f'btag_output_{year}.pickle', 'rb') as handle:
    output = pickle.load(handle)

In [8]:
f_out = uproot.recreate(f"bTagEff_{year}.root")

for h_name, histo in output.items():
    print('Writing', h_name)
    f_out[h_name] = histo['TT'].sum('dataset').to_hist()

f_out.close()

Writing h_num_b
Writing h_den_b
Writing h_num_c
Writing h_den_c
Writing h_num_l
Writing h_den_l
